In [1]:
import pickle
import pandas as pd
import time
from datasketch import MinHash, MinHashLSHForest
import re
import numpy as np

In [2]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [3]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['Tags']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [4]:
with open('Tags.pickle','rb') as f:
    forest = pickle.load(f)

In [5]:
num_recommendations = 2
db = pd.read_csv('tags.csv')
db['text'] = db['Stories'] + ' ' + db['Tags']
text = ''' View exams that I have scheduled including both exams that I have completed and exams that have yet to be completed Reschedule or cancel a reserved exam time up to 2Education hours in advance of the scheduled time Cancel a scheduled exam in one location and schedule in a different one, regardless of whether it is the third-party vendor's location or a Red Hat location Have my exam eligibility and scheduling status accurately captured in Red Hat's Seertech Learning Management System, where applicable'''
result = predict(text, db, 128 , num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.00697016716003418 seconds to query forest.

 Top Recommendation(s) is(are) 
 5    Modify
7     Admin
Name: Tags, dtype: object
